In [1]:
# pandas
import pandas as pd 
# pip install eurostat  #herramienta para importar datos de eurostat

In [2]:
import PySimpleGUI as sg
def selection(option):
    #sg.theme('DarkAmber')   # Add a touch of color
    #select the variable we want to calculate
    options = option

    # All the stuff inside your window.
    layout = [ 
            [sg.Text('Select one->'), sg.Listbox(options,select_mode=sg.LISTBOX_SELECT_MODE_SINGLE,size=(20,len(options)))],
            [sg.Button('Ok'), sg.Button('Cancel')]
        ]

    # Create the Window
    window = sg.Window('Make your choice', layout)

    # Event Loop to process "events" and get the "values" of the input
    while True:
        event, values = window.read()
        print( f"event={event}" )
        if event is None or event == 'Ok' or event == 'Cancel': # if user closes window or clicks cancel
            break
        
    # close  the window        
    window.close()

    if event == "Cancel":
        print( "You cancelled" )
    else:
        print('You entered ', values[0])
        sg.popup( f"You selected {values[0]}" )        
    return values

In [3]:
#change the number to select another database
database_sel= 'env_ac_taxind2' 
database_sel

'env_ac_taxind2'

In [4]:
#nama_10_a64 National accounts aggregates by industry (up to NACE A*64)
#NAMA_10_A64_P5 Gross capital formation by industry (up to NACE A*64)
#National accounts employment data by industry (up to NACE A*64)

import eurostat
df = eurostat.get_data_df(database_sel, flags=False)
df

,tax,unit,nace_r2,geo\time,2020,2019,2018,2017,2016,2015,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,ENV,MIO_EUR,A,AT,NaN,127.65,126.21,131.16,128.83,129.28,...,253.10,242.04,237.53,226.22,218.58,221.86,207.72,214.61,199.1,218.83
1,ENV,MIO_EUR,A,BE,NaN,184.79,188.51,178.49,171.41,114.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENV,MIO_EUR,A,BG,NaN,65.62,60.69,58.36,59.45,63.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENV,MIO_EUR,A,CH,NaN,279.36,265.82,279.41,270.21,259.73,...,133.37,130.78,131.41,136.48,135.58,NaN,NaN,NaN,NaN,NaN
4,ENV,MIO_EUR,A,CY,NaN,4.72,4.67,4.43,2.65,2.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26503,TRA,MIO_NAC,U,SE,NaN,0.02,0.02,0.03,0.03,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26504,TRA,MIO_NAC,U,SI,0.0,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26505,TRA,MIO_NAC,U,SK,NaN,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26506,TRA,MIO_NAC,U,TR,0.0,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index([     'tax',     'unit',  'nace_r2', 'geo\time',       2020,       2019,
             2018,       2017,       2016,       2015,       2014,       2013,
             2012,       2011,       2010,       2009,       2008,       2007,
             2006,       2005,       2004,       2003,       2002,       2001,
             2000,       1999,       1998,       1997,       1996,       1995],
      dtype='object')

In [6]:
#eliminate columns for years previous to 2011
years =[]
year_start=1995
year_study=2011
for y in range(year_start,year_study): 
    years.append(y)
df=df.drop(years, axis=1)#quitamos todas las columnas de añosque no nos interesan
#rename geo time column to be easier to handle because of the name
df.rename(columns={"geo\\time": "geo"}, inplace=True)
#We can get the unique values of the different variables

In [7]:
dic_unit = eurostat.get_dic('unit') #dictionary for all the unit in eurostat
#print the units in the database
for u in df.unit.unique():
    if u in dic_unit:
        print (u + " -"+dic_unit[u])
#select the unit that we want
unit=selection(df.unit.unique())[0]

MIO_EUR -Million euro
MIO_NAC -Million units of national currency
event=Ok
You entered  ['MIO_EUR']


In [8]:
dic_tax = eurostat.get_dic('tax') #dictionary for all the unit in eurostat
for item in df.tax.unique():
    if item in dic_tax:
        print (item + " -"+dic_tax[item])
tax=selection(df.tax.unique())[0]

ENV -Total environmental taxes
NRG -Energy taxes
POL -Pollution taxes
RES -Resource taxes
TRA -Transport taxes
event=Ok
You entered  ['ENV']


In [9]:
nace_tax=df.nace_r2.unique()

In [10]:
weights_target = pd.read_excel("ecosystemweights.xlsx")

nace_weights= weights_target.NACE_R2.unique()
nace_weights

array(['A', 'C10-C12', 'C13-C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29',
       'C30', 'C31', 'C32', 'C33', 'D', 'E36', 'E37-E39', 'F', 'G45',
       'G46', 'G47', 'H49', 'H50', 'H51', 'H52', 'H53', 'I', 'J58',
       'J59_J60', 'J61', 'J62_J63', 'L', 'M69_M70', 'M71', 'M72', 'M73',
       'M74_M75', 'N77', 'N78', 'N79', 'N80', 'N81', 'N82', 'P', 'Q86',
       'Q87_Q88', 'R90-R92', 'R93', 'S94', 'S95', 'S96', 'T'],
      dtype=object)

In [11]:
nace_dif=set(nace_weights)-set(nace_tax)
nace_dif

{'C31', 'C32', 'N80', 'N81', 'N82'}

In [12]:
nace_drop=set(nace_tax)-set(nace_weights)
nace_drop

{'A01',
 'A02',
 'A03',
 'B',
 'C',
 'C31_C32',
 'E',
 'EP_HH',
 'EP_NRES',
 'G',
 'H',
 'I-U',
 'K64',
 'K65',
 'K66',
 'N80-N82',
 'NAL',
 'O',
 'TOTAL',
 'TOTAL_HH_NRES',
 'U'}

In [13]:
#we elinimate al the codes in nace_drop except for 'C31_C32','N80-N82', which we have to disaggregate
nacedrop=['A01',
 'A02',
 'A03',
 'B',
 'C',
 'E',
 'EP_HH',
 'EP_NRES',
 'G',
 'H',
 'I-U',
 'K64',
 'K65',
 'K66',
 'NAL',
 'O',
 'TOTAL',
 'TOTAL_HH_NRES',
 'U']

In [14]:
countries_not_selected= ['UK', 'TR', 'AL','IS','LI','NO','CH','UK','ME','MK','AL','RS','TR','BA','EU28','EU15','EA','EA19','EA12']
#We can get the selected values we want from nace 
#naceall=df.nace_r2.unique()
#nace=set(naceall)-set(nacedrop)
#df[df.unit.isin(unit)&df.nace_r2.isin(nace)&df.na_item.isin(naitem)]
#or we can select those that are not in the list nacedrop by using ~df.nace_r2.isin(nacedrop) later

#we select the rows that have the right unit, nace codes, items and countries
df_selection=df[df.unit.isin(unit)&~df.nace_r2.isin(nacedrop)&df.tax.isin(tax)&~df.geo.isin(countries_not_selected)]
df_selection

,tax,unit,nace_r2,geo,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011
0,ENV,MIO_EUR,A,AT,NaN,127.65,126.21,131.16,128.83,129.28,129.46,129.09,130.71,130.61
1,ENV,MIO_EUR,A,BE,NaN,184.79,188.51,178.49,171.41,114.60,85.63,48.88,47.15,44.30
2,ENV,MIO_EUR,A,BG,NaN,65.62,60.69,58.36,59.45,63.95,60.74,62.88,58.73,56.47
4,ENV,MIO_EUR,A,CY,NaN,4.72,4.67,4.43,2.65,2.99,2.47,2.30,2.47,2.54
5,ENV,MIO_EUR,A,CZ,NaN,169.66,163.30,169.60,170.76,169.04,167.09,151.48,154.58,159.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2507,ENV,MIO_EUR,T,PT,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2508,ENV,MIO_EUR,T,RO,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2510,ENV,MIO_EUR,T,SE,NaN,1.63,1.62,0.53,0.00,0.00,0.00,0.00,0.00,0.00
2511,ENV,MIO_EUR,T,SI,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
#ratios to divide N80_82 and 31_32. Value Added from SBS
ratios_values = pd.read_excel("ratioCN-Value added at factor cost - million euro.xlsx")
ratios_values.set_index('nace_r2', inplace=True)#nace como índice
ratios_values

,Value added at factor cost - million euro2011,Value added at factor cost - million euro2012,Value added at factor cost - million euro2013,Value added at factor cost - million euro2014,Value added at factor cost - million euro2015,Value added at factor cost - million euro2016,Value added at factor cost - million euro2017,Value added at factor cost - million euro2018,Value added at factor cost - million euro2019
nace_r2,,,,,,,,,
C31,0.398512,0.403322,0.400551,0.398158,0.400751,0.409175,0.388565,0.382716,0.390244
C32,0.601488,0.596678,0.599449,0.601842,0.599249,0.590825,0.611435,0.617284,0.609756
N80,0.165778,0.162820,0.158483,0.152327,0.157963,0.160093,0.155610,0.145835,0.145557
N81,0.462572,0.459438,0.466741,0.463845,0.467105,0.467350,0.469511,0.477816,0.490550
N82,0.371650,0.377742,0.374777,0.383828,0.374932,0.372558,0.374879,0.376349,0.363893


In [16]:
# columns of countries to be eliminated
cols = ['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'EL', 'ES', 'FI','FR', 'HR', 'HU', 'IE', 'IT', 'LT','LU', 'LV', 'MT','NL', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']

#create a dataframe for the results
results_compiled = pd.DataFrame()
variable_of_interest="env taxes"

import numpy as np
#loop to calculate values for years
for j in range (2011,2019):
    year=j
    #we transpose the values for the year selected
    for i in range (year,year+1):
        df_target = pd.pivot_table(df_selection, values=i, index=['nace_r2'],columns=["geo"], aggfunc=np.sum, fill_value=0)#reorde
    df_target['sum'] = df_target[cols].sum(axis=1)#sums the columns of the countries to "sum" 
    df_target=df_target.drop(cols, axis=1)#eliminamos las columnas sumadas de países
    df_target.loc[df_target['EU27_2020']==0,'EU27_2020'] = df_target['sum']#now we can locate columns with 0 and make the substitution with the column "sum"
    moddf_target=df_target.drop('sum', axis=1)#we create a new table 
    #add rows C31,... and assign value with the ratio multiplying by the value of position C31_32
    year_ratio=j-2011
    moddf_target.loc['C31'] = ratios_values.iloc[0][year_ratio]*moddf_target.loc['C31_C32','EU27_2020']
    moddf_target.loc['C32'] = ratios_values.iloc[1][year_ratio]*moddf_target.loc['C31_C32','EU27_2020']
    moddf_target.loc['N80'] = ratios_values.iloc[2][year_ratio]*moddf_target.loc['N80-N82','EU27_2020']
    moddf_target.loc['N81'] = ratios_values.iloc[3][year_ratio]*moddf_target.loc['N80-N82','EU27_2020']
    moddf_target.loc['N82'] = ratios_values.iloc[4][year_ratio]*moddf_target.loc['N80-N82','EU27_2020']
    #we eliminate rows C31_C32
    moddf_target = moddf_target.drop(['C31_C32', 'N80-N82'])
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    #cargamos los pesos en el dataframe weight  
    weights_target = pd.read_excel("ecosystemweights.xlsx")
    weights_target.set_index('NACE_R2', inplace=True)#nace como índice
    column_name=str(variable_of_interest)+str(year)
    weights_target[variable_of_interest]=weights_target.index.map(moddf_target['EU27_2020'])#busca en df_target el nace y copia el valor de EU27_2020 en la fila del nace the weights_target
    #creamos la matriz de resultados
    results=weights_target
    for i in range(0,14):
        results.iloc[:, i]=weights_target.iloc[:, i]*weights_target[variable_of_interest]#we multiply the total value added by the weight for each nace
    results.loc['total'] = results.select_dtypes(pd.np.number).sum() #we add the values of each colum in a row call total
    results_compiled[column_name]=results.loc['total']
results_compiled = results_compiled.drop(variable_of_interest)
#sum of the totals
results_SMEs = results_compiled.copy()  
results_big = results_compiled.copy()
#results_compiled.loc['total'] = results_compiled.select_dtypes(pd.np.number).sum()
results_compiled  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


,env taxes2011,env taxes2012,env taxes2013,env taxes2014,env taxes2015,env taxes2016,env taxes2017,env taxes2018
Aerospace & Defence,2260.6155,2303.9871,2320.6128,2309.6778,2385.3279,2363.0260,2441.8646,2513.2734
Agri-food,10364.0442,10869.9259,11227.9056,11614.0866,12061.2748,12377.0364,12866.8156,13072.7308
Construction,11638.9843,11716.3651,11739.5042,11371.4942,11592.1810,12151.1553,12615.2437,13132.8895
Cultural and Creative Industries,2521.4778,2559.5350,2614.3051,2647.8557,2742.2428,2869.8284,2944.1222,3043.8518
Digital,2258.1293,2310.3114,2326.2444,2347.4496,2394.7096,2556.5716,2753.8680,2862.8868
Electronics,704.7252,743.5080,743.8706,775.3574,801.4584,880.4584,879.6082,899.7823
Energy - Renewables,2818.9795,3223.9444,3661.3752,3817.9437,4077.8447,3888.0763,3966.4223,4376.8129
Energy Intensive Industries,15718.4908,15674.2942,15954.8172,16303.2749,16432.0488,17031.5272,17366.0445,19271.7472
Health,4470.2925,4504.3077,4642.5003,4945.0841,4953.1361,5604.4581,5608.1561,5673.1658
Mobility - Transport - Automotive,18852.6884,18561.4382,19146.3250,18981.7551,18796.4300,19651.1065,20510.0726,21642.1731


In [17]:
#save to file
filename = "%s.xlsx" % variable_of_interest
results_compiled.to_excel(filename)